In [3]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        初始化数据集。
        :param root_dir: 包含图像文件的根目录。
        :param transform: 应用于图像的可选变换。
        """
        self.root_dir = os.path.join(root_dir, '')
        self.transform = transform
        self.images = []
        self.labels = []

        # 遍历目录，收集图像路径和标签
        for filename in os.listdir(root_dir):
            if filename.endswith('.json'):  # 假设图像文件后缀为.jpg
                img_path = os.path.join('/media/liushilei/DatAset/workspace/test/torch/data/nyc/cut_data', os.path.basename(filename).split('.')[0] + '.png')
                filename = os.path.join('/media/liushilei/DatAset/workspace/test/torch/data/labels/annotation_seq', filename)
                self.images.append(img_path)
                self.labels.append(filename)

    def __len__(self):
        """
        返回数据集中的图像数量。
        """
        return len(self.images)

    def __getitem__(self, idx):
        """
        根据索引获取一个图像和它的标签。
        """
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')  # 确保图像是RGB格式

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# 创建数据集的变换
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图像大小
    transforms.ToTensor(),  # 转换为Tensor
])

# 创建数据集实例
dataset = CustomDataset(root_dir='data/labels/annotation_seq', transform=transform)

# 现在可以使用PyTorch的DataLoader来加载数据集
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

### 可视化地图

In [ ]:
import json, os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def draw_lines_on_image(image_path, json_path):
    # 读取图像
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # 读取JSON文件
    with open(json_path, 'r') as file:
        data = json.load(file)

    # 遍历JSON中的每个元素
    for item in data:
        seq = item.get("seq", [])
        # 将seq中的点连成线
        if len(seq) > 1:
            for i in range(len(seq) - 1):
                start_point = tuple(seq[i])
                end_point = tuple(seq[i + 1])
                draw.line([start_point, end_point], fill='red', width=3)

    # 可视化图像
    plt.imshow(image)
    plt.axis('off')  # 不显示坐标轴
    plt.show()

# 示例用法
json_path = '/media/liushilei/DatAset/workspace/test/torch/data/labels/annotation_seq/002232_34.json'  # 替换为你的JSON文件路径
image_path = os.path.join('/media/liushilei/DatAset/workspace/test/torch/data/nyc/cut_data', os.path.basename(json_path).split('.')[0] + '.png')  # 替换为你的图像路径

draw_lines_on_image(image_path, json_path)

### 加载模型